In [15]:
import numpy as np

path_prefix = "./all/data/"
word_to_index = {} # map word to index
g_word_list = [] # sorted by frequence low to high
total_word_dict = {}
skip_words = set(['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'as', 'an', 'and', 'are', 'at', 'by', 'down', 'for', 'from', 'front', 'has', 'in', 'is', 'it', 'near', 'next', 'of', 'on', 'out', 'over', 'several', 'sits', 'some', 'that', 'the', 'there', 'through', 'to', 'top', 'up', 'while', 'with'])
train_des = [] #0/1/2 array of word
test_des = [] #0/1/2 array of word
train_feature_list = [] # list of feature_array by index
test_feature_list = []
all_predict_test_des = []
train_g_category_tag = []
train_g_item_tag = []
test_g_category_tag = []
test_g_item_tag = []
MAX_INPUT = 10000
MAX_OUTPUT = 2000
COMPONENTS = 400 # need to run again
PCA_NUMBER = 2000 # need to run again
ITER = 500

In [16]:
def read_file_to_string(file_name, keep_colon):
    f = open(file_name, "r")
    content = f.read()
    f.close()
    content = content.lower()
    content = content.replace(",", " ").replace(".", " ").replace("'", " ").replace("\"", " ")
    if (keep_colon == False):
        content = content.replace(":", " ")
    return content

def read_file_to_total_word_dict(file_name, keep_colon):
    word_list = read_file_to_string(file_name, keep_colon).split()
    for name in word_list:
        if name in skip_words:
            continue
        if name in total_word_dict:
            total_word_dict[name] += 1
            continue
        else:
            total_word_dict[name] = 1
            continue

def print_word_in_array(a):
    ret = ""
    for i in range(len(a)):
        if a[i] != 0:
            ret = ret + str(a[i]) + g_word_list[i] + " "
    print(ret)

In [17]:
total_word_dict = {}
for i in range(MAX_INPUT):
    read_file_to_total_word_dict(path_prefix+"descriptions_train/"+str(i)+".txt", False)
    read_file_to_total_word_dict(path_prefix+"tags_train/"+str(i)+".txt", True)

In [18]:
import functools
def cmp(a, b):
    a_num = abs(float(a.split("___")[0]))
    b_num = abs(float(b.split("___")[0]))
    if (a_num < b_num):
        return -1
    return 1

In [19]:
# load g_word_list
g_word_list = []
word_to_index = {}
tmp_list = []
#TODO: maybe try filter on number
for word in total_word_dict:
    if total_word_dict[word] <= 0:
        continue
    tmp_list.append(str(total_word_dict[word])+"___"+word)

tmp_list = sorted(tmp_list, key=functools.cmp_to_key(cmp))

for i in range(len(tmp_list)):
    word = tmp_list[i].split("___")[1]
    g_word_list.append(word)
    word_to_index[word] = i

In [20]:
# read to tags
train_g_category_tag = [0] * MAX_INPUT
train_g_item_tag = [0] * MAX_INPUT
test_g_category_tag = [0] * MAX_OUTPUT
test_g_item_tag = [0] * MAX_OUTPUT

#TODO: add tags in descriptions
def is_in_tag(file_number, word, train):
    if train:
        return (word in train_g_category_tag[file_number]) or (word in train_g_item_tag[file_number])
    return (word in test_g_category_tag[file_number]) or (word in test_g_item_tag[file_number])

def read_tags(file_name, train):
    lines = read_file_to_string(file_name, True).split("\n")
    index = int(file_name.split("/")[-1].replace(".txt", ""))
    current_category_set = set([])
    current_item_set = set([])
    
    for line in lines:
        cat = line.split(":")[0].strip()
        if cat in word_to_index:
            current_category_set.add(cat)

        item = ''
        if (cat.strip() != line.replace(":", "").strip()):
            item = line.split(":")[1].strip()
        if item in word_to_index:
            current_item_set.add(item)
    if (train):
        train_g_category_tag[index] = current_category_set
        train_g_item_tag[index] = current_item_set
    else:
        test_g_category_tag[index] = current_category_set
        test_g_item_tag[index] = current_item_set
    

for i in range(MAX_INPUT):
    read_tags(path_prefix+"tags_train/"+str(i)+".txt", True)
for i in range(MAX_OUTPUT):
    read_tags(path_prefix+"tags_test/"+str(i)+".txt", False)

In [21]:
# where to make the change
# todo: remove count to match test output
# lenpi ci, yaoshi you dehua, jianfen
# also try norm

# count_max_weight = 2.0
# uniqueness_max_weight = 2.0
# TODO: add words in tag_item to pic
def my_weight_func(index, count, train, file_num):
    if (count <= 0):
        return 0
#     count_apply = 0
#     if is_in_tag(file_num, g_word_list[index], train):
#         count_apply = 1
#     elif count >= 3:
#         count_apply = 1
#     return 1 + (len(g_word_list)-index)*uniqueness_max_weight/len(g_word_list) + count_apply*count_max_weight

#     if count >= 5:
#         return count
#     if is_in_tag(file_num, g_word_list[index], train):
#         return 5
    return 1

def weight_occurance_array(occurance_array, train, file_num):
    ret = [0] * len(occurance_array)
    for index in range(len(occurance_array)):
        ret[index] = my_weight_func(index, occurance_array[index], train, file_num)
    return ret

def read_des_to_list(path, file_num):
    ret = []
    train = (path.find("train") != -1)
    for i in range(file_num):
        int_array = [0] * len(g_word_list)
        temp_list = read_file_to_string(path+str(i)+".txt", False).split()
        for word in temp_list:
            if not word in word_to_index:
                continue
            int_array[word_to_index[word]] += 1
        ret.append(weight_occurance_array(int_array, train, i))
    return ret

train_des = read_des_to_list(path_prefix+"descriptions_train/", MAX_INPUT)
test_des = read_des_to_list(path_prefix+"descriptions_test/", MAX_OUTPUT)

In [22]:
# add tag to desc list

#TODO: careful with tag number
def add_tag_to_train_des():
    for sentence_index in range(MAX_INPUT):
        array_to_add = [0] * len(g_word_list)
        for tag in train_g_item_tag[sentence_index]:
            array_to_add[word_to_index[tag]] = 1
        
        ret = train_des[sentence_index].copy()
        for word_index in range(len(g_word_list)):
            ret[word_index] = max(ret[word_index], array_to_add[word_index])
        train_des[sentence_index] = ret

def add_tag_to_test_des():
    for sentence_index in range(MAX_OUTPUT):
        array_to_add = [0] * len(g_word_list)
        for tag in test_g_item_tag[sentence_index]:
            array_to_add[word_to_index[tag]] = 1
        
        ret = all_predict_test_des[sentence_index].copy()
        print(all_predict_test_des[sentence_index])
        for word_index in range(len(g_word_list)):
            ret[word_index] = max(ret[word_index], array_to_add[word_index])
        all_predict_test_des[sentence_index] = ret


In [23]:
add_tag_to_train_des()

In [24]:
# load feature list
def load_feature_into_list(filename):
    f = open(filename, "r")
    content = f.readlines()
    f.close()
    train = filename.find("train")
    for line in content:
        name_index = int(line.split(".jpg,")[0].split("/")[1])
        feature_list = line.split(".jpg,")[1].split(",")
        if train != -1:
            if name_index >= MAX_INPUT:
                continue
            train_feature_list[name_index] = feature_list
        else:
            if name_index >= MAX_OUTPUT:
                continue
            test_feature_list[name_index] = feature_list

train_feature_list = [0] * MAX_INPUT
test_feature_list = [0] * MAX_OUTPUT
# load_feature_into_list(path_prefix+"features_train/features_resnet1000_train.csv")
# load_feature_into_list(path_prefix+"features_test/features_resnet1000_test.csv")
load_feature_into_list(path_prefix+"features_train/features_resnet1000intermediate_train.csv")
load_feature_into_list(path_prefix+"features_test/features_resnet1000intermediate_test.csv")

train_feature_list = np.asarray(train_feature_list, dtype=np.float64)
test_feature_list = np.asarray(test_feature_list, dtype=np.float64)

In [25]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components = PCA_NUMBER)
# pca.fit(train_des)
# train_des_pca = pca.transform(train_des)
# test_des_pca = pca.transform(test_des)
train_des_pca = np.asarray(train_des)
test_des_pca = np.asarray(test_des)

print(train_des_pca.shape)
print(test_des_pca.shape)

# pca2 = PCA(n_components = PCA_NUMBER)
# pca2.fit(train_feature_list)
# train_feature_list = pca2.transform(train_feature_list)
# test_feature_list = pca2.transform(test_feature_list)
train_feature_list = np.asarray(train_feature_list)
test_feature_list = np.asarray(test_feature_list)

print(train_feature_list.shape)
print(test_feature_list.shape)

from sklearn.cross_decomposition import PLSRegression
print("Done 0")
model = PLSRegression(COMPONENTS, max_iter=ITER)
print("Done 1")
model.fit(train_feature_list, train_des_pca)
print("Done 2")

(10000, 9900)
(2000, 9900)
(10000, 2048)
(2000, 2048)
Done 0
Done 1


C:\Users\ranji\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:77: UserWarning: Maximum number of iterations reached
  warnings.warn('Maximum number of iterations reached')


Done 2


In [26]:
print("Start 3")
all_predict_test_des = model.predict(test_feature_list)
print("Done 3")

Start 3
Done 3


In [27]:
add_tag_to_test_des()

[0.00039068 0.00039068 0.00039188 ... 0.25481488 0.         0.01829317]
[0.00148874 0.00148874 0.00263842 ... 0.20535961 0.         0.21424969]
[6.13923703e-04 6.13923703e-04 1.10122180e-03 ... 3.08805349e-02
 0.00000000e+00 6.19508848e-01]
[ 0.00181366  0.00181366 -0.00130097 ...  0.19642685  0.
  0.14371268]
[ 7.23436489e-04  7.23436489e-04  6.15061361e-04 ...  7.13720322e-01
  0.00000000e+00 -1.03645831e-01]
[ 0.00102108  0.00102108 -0.00087709 ...  0.14408809  0.
 -0.11707659]
[0.00159249 0.00159249 0.00422679 ... 0.56669561 0.         0.00453848]
[6.11947752e-04 6.11947752e-04 1.17506999e-03 ... 8.23954845e-02
 0.00000000e+00 7.04167724e-01]
[ 1.05484898e-03  1.05484898e-03 -4.16495521e-04 ...  5.35602896e-01
  0.00000000e+00 -1.10734802e-01]
[ 5.89893086e-06  5.89893086e-06 -2.44405441e-03 ...  6.99816033e-01
  0.00000000e+00  7.11174928e-02]
[ 0.00020326  0.00020326 -0.00204429 ...  0.11463974  0.
  0.13975026]
[ 1.80822457e-04  1.80822457e-04 -5.13947627e-04 ...  6.76687634e-01

[-7.00177750e-04 -7.00177750e-04 -1.46470405e-03 ...  8.91173391e-01
  0.00000000e+00  6.75189880e-01]
[ 3.61745935e-04  3.61745935e-04 -1.89072457e-03 ...  6.06097264e-02
  0.00000000e+00  8.90618823e-01]
[ 0.00217741  0.00217741 -0.00134727 ... -0.25217708  0.
  0.39138097]
[1.60482635e-04 1.60482635e-04 3.34674329e-03 ... 4.46659797e-01
 0.00000000e+00 3.55498116e-03]
[-0.00125768 -0.00125768 -0.00063992 ...  0.59690052  0.
 -0.05857415]
[ 0.00293526  0.00293526 -0.00082978 ...  0.13101852  0.
 -0.14163093]
[-1.40882892e-03 -1.40882892e-03  1.68761809e-04 ...  3.27560049e-01
  0.00000000e+00  4.81263568e-01]
[-0.00109947 -0.00109947 -0.00099187 ...  0.61158074  0.
 -0.06245206]
[-0.00034106 -0.00034106 -0.00110635 ...  0.14980579  0.
  0.33876841]
[ 0.00755134  0.00755134 -0.00078054 ...  0.3632466   0.
  0.41797617]
[-0.00110008 -0.00110008 -0.0016852  ...  0.47951154  0.
 -0.03870587]
[ 0.00412251  0.00412251 -0.00148495 ...  0.20214843  0.
  0.05256635]
[ 0.00343778  0.00343778 -

 -0.03910097]
[ 0.00046823  0.00046823 -0.00064516 ...  0.43809856  0.
  0.46149462]
[ 6.88861849e-04  6.88861849e-04 -3.43438412e-04 ... -6.07949016e-02
  0.00000000e+00  5.52345237e-01]
[ 1.90685504e-04  1.90685504e-04 -1.45967867e-03 ...  2.77729528e-01
  0.00000000e+00  6.43021698e-02]
[-0.00145977 -0.00145977  0.00213455 ...  0.09655388  0.
 -0.10959533]
[-5.27572565e-05 -5.27572565e-05 -1.43198307e-03 ...  1.67335976e-01
  0.00000000e+00  5.81589474e-01]
[ 1.15195489e-03  1.15195489e-03 -2.83143702e-03 ...  5.50395556e-01
  0.00000000e+00  2.48228414e-04]
[ 0.0011102   0.0011102  -0.00086596 ... -0.1278775   0.
  0.77690997]
[0.00032195 0.00032195 0.00062494 ... 0.07114495 0.         0.2766519 ]
[ 0.00218455  0.00218455 -0.00172845 ...  0.10597735  0.
  0.18909292]
[ 3.85153809e-04  3.85153809e-04 -6.28657712e-03 ...  4.51509922e-01
  0.00000000e+00 -9.94426521e-02]
[-0.00081475 -0.00081475  0.00238806 ...  0.49597571  0.
  0.30970419]
[0.00115649 0.00115649 0.00134661 ... 0.0871

[-1.63311233e-04 -1.63311233e-04  1.22535404e-03 ...  1.75478699e-01
  0.00000000e+00  6.45734412e-01]
[ 0.00223913  0.00223913 -0.00222085 ...  0.31009857  0.
  0.06233178]
[ 0.00057307  0.00057307 -0.00512552 ...  0.41152884  0.
 -0.06589478]
[0.00144374 0.00144374 0.0016205  ... 0.40253538 0.         0.08285271]
[ 0.00037273  0.00037273 -0.00031592 ...  0.27610543  0.
  0.05818018]
[0.00082564 0.00082564 0.00072116 ... 0.1673746  0.         0.26896843]
[ 0.00054605  0.00054605  0.00037366 ...  0.05669018  0.
 -0.10077511]
[-0.00152013 -0.00152013 -0.00019657 ...  0.05582599  0.
 -0.1409499 ]
[0.00037616 0.00037616 0.00138307 ... 0.05510684 0.         0.1228385 ]
[8.25846397e-04 8.25846397e-04 2.58829691e-04 ... 2.96098160e-01
 0.00000000e+00 4.49566965e-01]
[0.00145974 0.00145974 0.00305384 ... 0.08224067 0.         0.5582328 ]
[0.00041643 0.00041643 0.00032612 ... 0.07391921 0.         0.04057781]
[-0.00055062 -0.00055062  0.00322396 ...  0.19642234  0.
  0.02307656]
[ 0.00122119  

[-0.00084798 -0.00084798 -0.0008022  ...  0.10345157  0.
  0.00842212]
[-0.00361527 -0.00361527  0.00155648 ...  0.45234199  0.
  0.24620261]
[-0.00158223 -0.00158223  0.00082299 ...  0.04217712  0.
  0.1269343 ]
[-0.00264829 -0.00264829  0.00189914 ... -0.05879409  0.
  0.02526919]
[ 0.00557055  0.00557055 -0.00072693 ...  0.15161605  0.
  0.20593106]
[ 0.00433579  0.00433579 -0.00247526 ...  0.14888436  0.
  0.10310732]
[0.00220363 0.00220363 0.00263535 ... 0.36237339 0.         0.49349035]
[ 0.00214525  0.00214525 -0.0025693  ...  0.05138523  0.
  0.53497456]
[ 0.00283263  0.00283263  0.00462422 ...  0.74170013  0.
 -0.03997917]
[ 1.46283992e-03  1.46283992e-03  6.28969921e-04 ...  8.45381861e-01
  0.00000000e+00 -8.21177523e-02]
[-0.00333809 -0.00333809  0.00412737 ...  0.22579345  0.
 -0.05374634]
[2.38945211e-03 2.38945211e-03 5.86875017e-05 ... 1.30192507e-01
 0.00000000e+00 3.59870155e-01]
[-0.0003282  -0.0003282  -0.00247174 ...  0.18305146  0.
  0.10865104]
[ 5.90789952e-04  

...

[0.00187559 0.00187559 0.00258346 ... 0.31467094 0.         0.43557598]
[ 1.76327078e-04  1.76327078e-04 -7.64609118e-04 ...  2.66420038e-01
  0.00000000e+00  3.31411896e-01]
[-0.00020197 -0.00020197  0.00541689 ...  0.17181412  0.
  0.052558  ]
[ 1.92318075e-04  1.92318075e-04 -3.04345593e-03 ...  3.09285531e-01
  0.00000000e+00 -7.48924705e-02]
[ 1.05693913e-03  1.05693913e-03 -3.67240245e-04 ...  7.10916405e-01
  0.00000000e+00  4.70976086e-01]
[ 0.00484004  0.00484004 -0.00099843 ...  0.11290742  0.
  0.6618241 ]
[0.00063237 0.00063237 0.0015869  ... 0.44626849 0.         0.06264734]
[ 0.0003633   0.0003633  -0.00072701 ...  0.3447513   0.
  0.02463275]
[-4.20557691e-04 -4.20557691e-04  1.66471693e-03 ... -5.61211345e-02
  0.00000000e+00  8.81071004e-01]
[0.00198362 0.00198362 0.00086425 ... 0.59286275 0.         0.10357954]
[ 0.00456854  0.00456854 -0.00243662 ...  0.40258506  0.
  0.12064363]
[-0.00274806 -0.00274806 -0.00406622 ...  0.52151266  0.
  0.59627771]
[ 1.92255583e-04 

[-0.00179368 -0.00179368  0.01222403 ...  0.26388899  0.
  0.27420828]
[ 5.60422995e-04  5.60422995e-04  1.54981516e-04 ...  4.22117176e-01
  0.00000000e+00 -1.44918393e-01]
[-0.0016711  -0.0016711  -0.00166016 ...  0.4345232   0.
 -0.00384957]
[ 0.00056841  0.00056841 -0.00053164 ...  0.52153011  0.
  0.21518125]
[-0.00194124 -0.00194124 -0.00281316 ...  0.69453333  0.
  0.05990613]
[0.00062861 0.00062861 0.0036382  ... 0.3980912  0.         0.152005  ]
[ 3.83642352e-03  3.83642352e-03 -1.58611182e-04 ...  4.25811531e-01
  0.00000000e+00  9.73642418e-02]
[ 0.00286372  0.00286372  0.00110232 ...  0.39952177  0.
 -0.16912284]
[-0.00234094 -0.00234094 -0.00242765 ...  0.01999888  0.
  0.77852798]
[ 0.00084275  0.00084275 -0.00121804 ...  0.14603278  0.
  0.07480896]
[-4.39810136e-04 -4.39810136e-04 -1.92045973e-03 ...  4.93871244e-01
  0.00000000e+00  4.10441350e-01]
[-7.86481598e-05 -7.86481598e-05  8.18598885e-04 ...  2.64448001e-01
  0.00000000e+00  3.29372580e-01]
[-0.00408243 -0.004

[-0.00300355 -0.00300355  0.00065737 ...  0.44657076  0.
  0.31945699]
[-0.00065234 -0.00065234 -0.00157738 ...  0.29940627  0.
  0.01893687]
[-0.00219023 -0.00219023  0.00206225 ...  0.3032082   0.
  0.11330785]
[ 0.00466008  0.00466008 -0.00090979 ...  0.22514111  0.
  0.25655673]
[0.0015265  0.0015265  0.00216676 ... 0.36490503 0.         0.241788  ]
[0.00262266 0.00262266 0.00162438 ... 0.43431194 0.         0.01463287]
[ 0.00057629  0.00057629  0.00094225 ...  0.38892377  0.
 -0.03959919]
[-0.00154688 -0.00154688 -0.00056404 ...  0.11925277  0.
  0.48870397]
[-0.00136464 -0.00136464  0.00761877 ...  0.09441549  0.
  0.08401541]
[-0.00068349 -0.00068349 -0.00311233 ...  0.21885741  0.
  0.2409959 ]
[-0.00116551 -0.00116551  0.00063728 ...  0.22486454  0.
  0.56068773]
[4.48725472e-04 4.48725472e-04 3.78866222e-03 ... 7.86650797e-02
 0.00000000e+00 6.50002226e-01]
[ 0.00043462  0.00043462  0.00265406 ... -0.06246895  0.
  0.32260828]
[-0.00056863 -0.00056863  0.003764   ...  0.50821

  0.57556516]
[ 0.00068867  0.00068867 -0.00221756 ... -0.013041    0.
 -0.05799137]
[ 1.15050159e-03  1.15050159e-03 -1.84878349e-04 ...  4.45195899e-01
  0.00000000e+00  1.63862594e-01]
[ 0.00274819  0.00274819 -0.00242305 ...  0.3099215   0.
 -0.18407106]
[ 0.00177685  0.00177685  0.00066688 ...  0.50786703  0.
 -0.20061531]
[-0.00214134 -0.00214134  0.00455215 ...  0.29995084  0.
  0.11095297]
[ 0.00138519  0.00138519 -0.00086442 ...  0.38760671  0.
  0.22074816]
[ 0.00483117  0.00483117 -0.00220214 ...  0.27308661  0.
  0.08134694]
[-0.00151789 -0.00151789  0.00183281 ...  0.42962626  0.
 -0.16153948]
[ 0.00136691  0.00136691  0.00326281 ...  0.06946875  0.
 -0.06661511]
[-1.54911577e-04 -1.54911577e-04 -3.01042762e-03 ...  3.70990001e-01
  0.00000000e+00 -8.91528945e-03]
[ 0.00088999  0.00088999 -0.00133975 ...  0.02282372  0.
  0.19556799]
[7.12383468e-04 7.12383468e-04 2.00289086e-04 ... 2.51808592e-02
 0.00000000e+00 2.20262810e-01]
[0.00200125 0.00200125 0.00711891 ... 0.5716

[ 0.00146707  0.00146707 -0.0011431  ...  0.13974095  0.
  0.67394038]
[0.00102011 0.00102011 0.00055341 ... 0.23728513 0.         0.24590781]
[ 0.00129713  0.00129713 -0.001633   ...  0.23367589  0.
  0.35304769]
[0.00269714 0.00269714 0.00041914 ... 0.37243893 0.         0.03771517]
[0.00247206 0.00247206 0.00127799 ... 0.17332957 0.         0.23069157]
[ 1.06810106e-03  1.06810106e-03 -9.74564038e-05 ...  4.38040266e-01
  0.00000000e+00  1.57688091e-02]
[ 0.00128906  0.00128906 -0.00266941 ...  0.34682928  0.
  0.13268469]
[-2.37341163e-03 -2.37341163e-03  3.32804713e-04 ...  3.46210359e-01
  0.00000000e+00  5.29223444e-02]
[-0.00109144 -0.00109144  0.0021844  ...  0.68685207  0.
 -0.01071344]


In [28]:
import sklearn.metrics

# return a score
def compare_feature(target, given):
    return float(sklearn.metrics.pairwise.cosine_distances([np.array(target, dtype=float)], [np.array(given, dtype=float)]))
#     return float(np.linalg.norm(np.array(target, dtype=float) - np.array(given, dtype=float)))

def get_top_20(target):
    ret = []
    for index in range(MAX_OUTPUT):
        given = all_predict_test_des[index]
        score = compare_feature(target, given)
        item = str(score) + "___" + str(index)
        ret.append(item)
    return sorted(ret, key=functools.cmp_to_key(cmp))[:20]

# f = open("reverse_count_"+str(count_max_weight)+"unique"+str(uniqueness_max_weight)+".csv", "w")
f = open("reverse_count_weight_canary.csv", "w")
f.write("Descritpion_ID,Top_20_Image_IDs\n")
for index in range(MAX_OUTPUT):
    target_desc = test_des_pca[index]
    twenty_image_list = get_top_20(target_desc)
    string_to_write = "" + str(index) + ".txt,"
    for name in twenty_image_list:
        file_name = name.split("___")[1] + ".jpg"+" "
        string_to_write += file_name
    string_to_write = string_to_write.strip() + "\n"
    print(string_to_write)
    f.write(string_to_write)

f.close()

0.txt,1105.jpg 1702.jpg 93.jpg 1420.jpg 1648.jpg 860.jpg 338.jpg 714.jpg 1107.jpg 1574.jpg 561.jpg 1480.jpg 1135.jpg 1344.jpg 292.jpg 1033.jpg 700.jpg 375.jpg 255.jpg 1962.jpg

1.txt,833.jpg 589.jpg 818.jpg 1529.jpg 457.jpg 1116.jpg 1807.jpg 80.jpg 508.jpg 169.jpg 809.jpg 1742.jpg 1071.jpg 314.jpg 1156.jpg 948.jpg 1820.jpg 1143.jpg 331.jpg 1199.jpg

2.txt,1724.jpg 1840.jpg 598.jpg 649.jpg 1030.jpg 1208.jpg 838.jpg 1421.jpg 159.jpg 1713.jpg 332.jpg 445.jpg 1665.jpg 1084.jpg 1059.jpg 1437.jpg 42.jpg 1366.jpg 1471.jpg 1866.jpg

3.txt,1101.jpg 1024.jpg 26.jpg 1514.jpg 770.jpg 1778.jpg 1215.jpg 1609.jpg 1035.jpg 1765.jpg 374.jpg 1620.jpg 207.jpg 469.jpg 1415.jpg 1207.jpg 1513.jpg 750.jpg 71.jpg 843.jpg

4.txt,742.jpg 292.jpg 1816.jpg 1263.jpg 922.jpg 594.jpg 413.jpg 1231.jpg 712.jpg 1203.jpg 1131.jpg 359.jpg 484.jpg 1187.jpg 1620.jpg 1269.jpg 1258.jpg 549.jpg 1105.jpg 1423.jpg

5.txt,771.jpg 51.jpg 492.jpg 1991.jpg 330.jpg 1693.jpg 1335.jpg 814.jpg 1874.jpg 703.jpg 1660.jpg 846.jpg 1426.jpg

47.txt,1767.jpg 223.jpg 148.jpg 1536.jpg 925.jpg 441.jpg 234.jpg 1928.jpg 1632.jpg 1149.jpg 986.jpg 174.jpg 817.jpg 874.jpg 1661.jpg 1133.jpg 340.jpg 1731.jpg 573.jpg 709.jpg

48.txt,445.jpg 1713.jpg 953.jpg 1866.jpg 1840.jpg 1724.jpg 159.jpg 634.jpg 1665.jpg 934.jpg 1033.jpg 1664.jpg 883.jpg 696.jpg 838.jpg 904.jpg 1183.jpg 979.jpg 1641.jpg 1242.jpg

49.txt,892.jpg 1743.jpg 1136.jpg 942.jpg 595.jpg 147.jpg 586.jpg 761.jpg 1131.jpg 1924.jpg 1135.jpg 1350.jpg 1396.jpg 697.jpg 1267.jpg 484.jpg 642.jpg 118.jpg 1936.jpg 1526.jpg

50.txt,183.jpg 1534.jpg 1190.jpg 1832.jpg 932.jpg 1290.jpg 560.jpg 1711.jpg 384.jpg 1432.jpg 415.jpg 1703.jpg 1223.jpg 91.jpg 1469.jpg 54.jpg 1302.jpg 886.jpg 1022.jpg 1363.jpg

51.txt,1727.jpg 1404.jpg 514.jpg 1215.jpg 26.jpg 1207.jpg 1379.jpg 1145.jpg 989.jpg 770.jpg 475.jpg 1263.jpg 1070.jpg 960.jpg 1962.jpg 1052.jpg 1734.jpg 1097.jpg 731.jpg 1889.jpg

52.txt,1926.jpg 346.jpg 1746.jpg 745.jpg 1798.jpg 1137.jpg 620.jpg 1846.jpg 914.jpg 1877.jpg 880.jpg 1236.jpg 

94.txt,1143.jpg 885.jpg 1039.jpg 835.jpg 331.jpg 1116.jpg 286.jpg 1807.jpg 1714.jpg 1408.jpg 1550.jpg 571.jpg 1313.jpg 515.jpg 1835.jpg 160.jpg 1196.jpg 14.jpg 1173.jpg 1689.jpg

95.txt,51.jpg 1335.jpg 492.jpg 814.jpg 771.jpg 1426.jpg 1693.jpg 846.jpg 1943.jpg 703.jpg 630.jpg 1913.jpg 526.jpg 330.jpg 1888.jpg 274.jpg 20.jpg 1991.jpg 1118.jpg 940.jpg

96.txt,1713.jpg 1840.jpg 445.jpg 1724.jpg 634.jpg 696.jpg 953.jpg 1665.jpg 904.jpg 838.jpg 1471.jpg 1366.jpg 159.jpg 1023.jpg 1084.jpg 883.jpg 579.jpg 1866.jpg 979.jpg 1242.jpg

97.txt,690.jpg 1664.jpg 1483.jpg 1033.jpg 1440.jpg 607.jpg 938.jpg 1605.jpg 319.jpg 445.jpg 1211.jpg 1008.jpg 1460.jpg 1487.jpg 1739.jpg 1562.jpg 399.jpg 1853.jpg 1508.jpg 1027.jpg

98.txt,899.jpg 712.jpg 414.jpg 1821.jpg 1584.jpg 1333.jpg 1734.jpg 1758.jpg 1655.jpg 1289.jpg 770.jpg 1377.jpg 549.jpg 1620.jpg 217.jpg 1020.jpg 996.jpg 1101.jpg 648.jpg 1311.jpg

99.txt,1683.jpg 830.jpg 1825.jpg 1417.jpg 1267.jpg 1685.jpg 1585.jpg 1374.jpg 1576.jpg 1640.jpg 966.jpg 172

141.txt,1127.jpg 37.jpg 1385.jpg 1098.jpg 2.jpg 31.jpg 1736.jpg 636.jpg 1696.jpg 1577.jpg 1568.jpg 670.jpg 1318.jpg 1676.jpg 1552.jpg 1236.jpg 52.jpg 1774.jpg 1415.jpg 1433.jpg

142.txt,890.jpg 1014.jpg 1304.jpg 251.jpg 713.jpg 301.jpg 1160.jpg 988.jpg 423.jpg 1951.jpg 380.jpg 1578.jpg 961.jpg 83.jpg 963.jpg 1863.jpg 1159.jpg 1299.jpg 1182.jpg 1549.jpg

143.txt,1713.jpg 445.jpg 953.jpg 1665.jpg 1724.jpg 1641.jpg 1124.jpg 1664.jpg 784.jpg 1033.jpg 634.jpg 1840.jpg 1366.jpg 1866.jpg 1208.jpg 1242.jpg 669.jpg 696.jpg 117.jpg 1030.jpg

144.txt,1077.jpg 1609.jpg 1024.jpg 717.jpg 843.jpg 1769.jpg 1720.jpg 749.jpg 1222.jpg 1553.jpg 943.jpg 1430.jpg 1185.jpg 1413.jpg 729.jpg 1382.jpg 397.jpg 1500.jpg 626.jpg 647.jpg

145.txt,692.jpg 1563.jpg 694.jpg 1246.jpg 513.jpg 1079.jpg 1869.jpg 1080.jpg 780.jpg 665.jpg 1537.jpg 278.jpg 1652.jpg 1811.jpg 1229.jpg 1983.jpg 945.jpg 396.jpg 1378.jpg 499.jpg

146.txt,111.jpg 1931.jpg 1987.jpg 1833.jpg 1556.jpg 1864.jpg 1618.jpg 1674.jpg 860.jpg 561.jpg 1648.j

188.txt,1104.jpg 1829.jpg 327.jpg 551.jpg 277.jpg 1583.jpg 283.jpg 1766.jpg 241.jpg 1467.jpg 1947.jpg 195.jpg 1390.jpg 447.jpg 1604.jpg 710.jpg 248.jpg 1387.jpg 1946.jpg 472.jpg

189.txt,1632.jpg 148.jpg 802.jpg 986.jpg 1930.jpg 709.jpg 1767.jpg 618.jpg 1403.jpg 325.jpg 85.jpg 925.jpg 474.jpg 852.jpg 71.jpg 161.jpg 547.jpg 573.jpg 1418.jpg 1996.jpg

190.txt,1640.jpg 1579.jpg 1744.jpg 1150.jpg 1859.jpg 1038.jpg 1685.jpg 1728.jpg 405.jpg 1683.jpg 173.jpg 1587.jpg 612.jpg 988.jpg 369.jpg 307.jpg 276.jpg 1578.jpg 1585.jpg 746.jpg

191.txt,1103.jpg 1213.jpg 141.jpg 210.jpg 498.jpg 1046.jpg 1384.jpg 734.jpg 1050.jpg 620.jpg 2.jpg 256.jpg 112.jpg 534.jpg 964.jpg 1087.jpg 1466.jpg 101.jpg 70.jpg 1856.jpg

192.txt,1143.jpg 885.jpg 331.jpg 1807.jpg 1039.jpg 1408.jpg 286.jpg 160.jpg 1134.jpg 1550.jpg 169.jpg 815.jpg 1806.jpg 121.jpg 553.jpg 571.jpg 1068.jpg 1196.jpg 14.jpg 296.jpg

193.txt,54.jpg 1210.jpg 1290.jpg 1363.jpg 1022.jpg 1832.jpg 1703.jpg 1775.jpg 1708.jpg 886.jpg 1123.jpg 1534.jpg 171

...

1811.txt,776.jpg 46.jpg 526.jpg 1696.jpg 1510.jpg 1076.jpg 829.jpg 357.jpg 20.jpg 1729.jpg 1481.jpg 1888.jpg 1166.jpg 619.jpg 82.jpg 814.jpg 1535.jpg 1161.jpg 1110.jpg 1913.jpg

1812.txt,1528.jpg 1534.jpg 183.jpg 560.jpg 1832.jpg 1223.jpg 415.jpg 1703.jpg 1363.jpg 384.jpg 1290.jpg 932.jpg 1190.jpg 1362.jpg 1432.jpg 91.jpg 1461.jpg 10.jpg 54.jpg 1022.jpg

1813.txt,16.jpg 1701.jpg 1944.jpg 582.jpg 262.jpg 1444.jpg 718.jpg 1458.jpg 897.jpg 1352.jpg 1151.jpg 1790.jpg 489.jpg 655.jpg 1540.jpg 1472.jpg 156.jpg 1275.jpg 1933.jpg 391.jpg

1814.txt,1403.jpg 1632.jpg 709.jpg 743.jpg 618.jpg 148.jpg 1767.jpg 573.jpg 71.jpg 1418.jpg 1511.jpg 1514.jpg 474.jpg 1571.jpg 925.jpg 986.jpg 1930.jpg 234.jpg 161.jpg 1248.jpg

1815.txt,320.jpg 1819.jpg 1450.jpg 779.jpg 1953.jpg 432.jpg 59.jpg 1795.jpg 661.jpg 677.jpg 1125.jpg 1279.jpg 944.jpg 452.jpg 549.jpg 650.jpg 764.jpg 1821.jpg 479.jpg 1463.jpg

1816.txt,1190.jpg 1290.jpg 1711.jpg 183.jpg 560.jpg 1832.jpg 1534.jpg 1123.jpg 932.jpg 54.jpg 91.jpg 1022.jp

1857.txt,874.jpg 65.jpg 1081.jpg 751.jpg 801.jpg 1861.jpg 1091.jpg 875.jpg 1784.jpg 1590.jpg 1675.jpg 1741.jpg 1372.jpg 1697.jpg 187.jpg 1051.jpg 1709.jpg 205.jpg 1619.jpg 851.jpg

1858.txt,278.jpg 1007.jpg 578.jpg 1478.jpg 396.jpg 1019.jpg 1349.jpg 385.jpg 1805.jpg 1554.jpg 1545.jpg 1490.jpg 1378.jpg 952.jpg 438.jpg 707.jpg 556.jpg 1994.jpg 671.jpg 827.jpg

1859.txt,241.jpg 1604.jpg 327.jpg 1747.jpg 1467.jpg 1946.jpg 195.jpg 1104.jpg 551.jpg 1829.jpg 277.jpg 710.jpg 1947.jpg 1390.jpg 664.jpg 404.jpg 283.jpg 477.jpg 581.jpg 1766.jpg

1860.txt,1552.jpg 203.jpg 1189.jpg 1637.jpg 413.jpg 933.jpg 716.jpg 275.jpg 1152.jpg 190.jpg 1879.jpg 1568.jpg 747.jpg 1301.jpg 1696.jpg 817.jpg 243.jpg 1423.jpg 1126.jpg 1831.jpg

1861.txt,1829.jpg 558.jpg 327.jpg 241.jpg 195.jpg 1947.jpg 277.jpg 1390.jpg 1244.jpg 1104.jpg 1747.jpg 551.jpg 1604.jpg 1467.jpg 1583.jpg 472.jpg 1766.jpg 248.jpg 1884.jpg 283.jpg

1862.txt,84.jpg 793.jpg 1626.jpg 935.jpg 1452.jpg 1309.jpg 1684.jpg 1268.jpg 762.jpg 387.jpg 232.j

1903.txt,1488.jpg 689.jpg 1961.jpg 1694.jpg 1249.jpg 1285.jpg 448.jpg 1085.jpg 168.jpg 1270.jpg 1155.jpg 987.jpg 1328.jpg 685.jpg 1239.jpg 481.jpg 1900.jpg 409.jpg 1857.jpg 794.jpg

1904.txt,833.jpg 1156.jpg 1753.jpg 1417.jpg 562.jpg 1529.jpg 948.jpg 589.jpg 758.jpg 1757.jpg 1907.jpg 1071.jpg 1489.jpg 875.jpg 1337.jpg 1372.jpg 1861.jpg 809.jpg 1590.jpg 435.jpg

1905.txt,1684.jpg 1268.jpg 1090.jpg 1072.jpg 1626.jpg 232.jpg 793.jpg 1245.jpg 806.jpg 102.jpg 84.jpg 209.jpg 1113.jpg 1658.jpg 1314.jpg 935.jpg 524.jpg 911.jpg 917.jpg 861.jpg

1906.txt,639.jpg 1948.jpg 1918.jpg 243.jpg 1568.jpg 719.jpg 1622.jpg 162.jpg 256.jpg 1817.jpg 1804.jpg 772.jpg 103.jpg 1.jpg 1350.jpg 1057.jpg 1261.jpg 1329.jpg 464.jpg 727.jpg

1907.txt,192.jpg 1547.jpg 843.jpg 633.jpg 1330.jpg 568.jpg 981.jpg 691.jpg 1553.jpg 55.jpg 943.jpg 1720.jpg 157.jpg 153.jpg 1960.jpg 1443.jpg 1024.jpg 676.jpg 224.jpg 1383.jpg

1908.txt,1286.jpg 599.jpg 246.jpg 50.jpg 93.jpg 1047.jpg 723.jpg 1410.jpg 678.jpg 305.jpg 202.jpg 226.j

1949.txt,1829.jpg 1390.jpg 277.jpg 551.jpg 248.jpg 327.jpg 1947.jpg 241.jpg 195.jpg 1104.jpg 1747.jpg 1583.jpg 1884.jpg 558.jpg 1467.jpg 1766.jpg 1244.jpg 1604.jpg 283.jpg 710.jpg

1950.txt,1346.jpg 1557.jpg 837.jpg 245.jpg 719.jpg 923.jpg 1397.jpg 1031.jpg 1543.jpg 1901.jpg 1847.jpg 1622.jpg 103.jpg 1389.jpg 1950.jpg 826.jpg 266.jpg 1948.jpg 1336.jpg 113.jpg

1951.txt,1621.jpg 128.jpg 1235.jpg 1012.jpg 64.jpg 1364.jpg 1813.jpg 853.jpg 269.jpg 708.jpg 688.jpg 439.jpg 194.jpg 544.jpg 1473.jpg 1376.jpg 777.jpg 214.jpg 1623.jpg 929.jpg

1952.txt,67.jpg 590.jpg 1810.jpg 1814.jpg 1688.jpg 1754.jpg 247.jpg 668.jpg 1391.jpg 701.jpg 1807.jpg 185.jpg 1326.jpg 149.jpg 1049.jpg 1039.jpg 874.jpg 6.jpg 169.jpg 470.jpg

1953.txt,544.jpg 708.jpg 1621.jpg 269.jpg 1813.jpg 1364.jpg 1012.jpg 439.jpg 128.jpg 1970.jpg 64.jpg 777.jpg 194.jpg 853.jpg 1984.jpg 1473.jpg 1297.jpg 1235.jpg 688.jpg 1623.jpg

1954.txt,1937.jpg 859.jpg 118.jpg 12.jpg 1481.jpg 1280.jpg 595.jpg 1743.jpg 729.jpg 1679.jpg 1342.jpg 653

1995.txt,1033.jpg 445.jpg 607.jpg 1724.jpg 1713.jpg 1840.jpg 1866.jpg 159.jpg 934.jpg 42.jpg 953.jpg 634.jpg 838.jpg 1962.jpg 511.jpg 1702.jpg 184.jpg 323.jpg 883.jpg 866.jpg

1996.txt,748.jpg 1853.jpg 1164.jpg 180.jpg 319.jpg 398.jpg 860.jpg 1027.jpg 399.jpg 1508.jpg 938.jpg 680.jpg 1605.jpg 1739.jpg 1211.jpg 1008.jpg 184.jpg 1460.jpg 1392.jpg 1962.jpg

1997.txt,104.jpg 657.jpg 1026.jpg 293.jpg 1163.jpg 58.jpg 1638.jpg 961.jpg 1266.jpg 782.jpg 1564.jpg 1407.jpg 342.jpg 1555.jpg 1061.jpg 816.jpg 1271.jpg 1773.jpg 11.jpg 876.jpg

1998.txt,483.jpg 206.jpg 1069.jpg 861.jpg 597.jpg 1117.jpg 1273.jpg 983.jpg 141.jpg 1451.jpg 911.jpg 922.jpg 1204.jpg 912.jpg 806.jpg 1050.jpg 832.jpg 562.jpg 1092.jpg 1892.jpg

1999.txt,729.jpg 1827.jpg 307.jpg 405.jpg 1106.jpg 1342.jpg 243.jpg 453.jpg 745.jpg 308.jpg 1167.jpg 1728.jpg 32.jpg 1260.jpg 1137.jpg 369.jpg 1846.jpg 285.jpg 653.jpg 772.jpg

